# Предсказание стоимости жилья

### Описание проекта

В проекте нам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году.

Каждая строка содержит агрегированную статистику о жилом массиве. Жилой массив — минимальная географическая единица с населением от 600 до 3000 человек в зависимости от штата. Одна строка в данных содержит статистику в среднем о 1425.5 обитателях жилого массива.

На основе данных нужно предсказать медианную стоимость дома в жилом массиве — median_house_value. Обучим модель и сделаем предсказания на тестовой выборке. Для оценки качества модели используем метрики RMSE, MAE и R2.

### Описание данных

В колонках датасета содержатся следующие данные:

    longitude — широта;
    latitude — долгота;
    housing_median_age — медианный возраст жителей жилого массива;
    total_rooms — общее количество комнат в домах жилого массива;
    total_bedrooms — общее количество спален в домах жилого массива;
    population — количество человек, которые проживают в жилом массиве;
    households — количество домовладений в жилом массиве;
    median_income — медианный доход жителей жилого массива;
    median_house_value — медианная стоимость дома в жилом массиве;
    ocean_proximity — близость к океану.

### План выполнения проекта

    Инициализируем локальную Spark-сессию.
    Прочитайтем содержимое файла /datasets/housing.csv.
    Выведем типы данных колонок датасета. Используем методы pySpark.
    Выполним предобработку данных:
        Исследуем данные на наличие пропусков и заполним их, выбрав значения по своему усмотрению.
        Преобразуем колонку с категориальными значениями техникой One hot encoding.
    Построим две модели линейной регрессии на разных наборах данных:
        используя все данные из файла;
        используя только числовые переменные, исключив категориальные.
        Для построения модели используем оценщик LinearRegression из библиотеки MLlib.
    Сравним результаты работы линейной регрессии на двух наборах данных по метрикам RMSE, MAE и R2.

## Подготовка и анализ данных

### EDA

Импорт необходимых библиотек и запуск spark сессии.

In [1]:
import pandas as pd 
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import Imputer
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression

pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder    
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator
        
SEED = 31416

spark = SparkSession.builder \
                    .master("local") \
                    .appName("California Housing ML") \
                    .getOrCreate()

Прочитайтем содержимое файла /datasets/housing.csv.

In [2]:
df = spark.read.load('/datasets/housing.csv', format='csv', sep=',', inferSchema=True, header='true')
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Выведем типы данных колонок датасета. Используем методы pySpark.

In [3]:
pd.DataFrame(df.dtypes, columns=['column', 'type'])

,column,type
0,longitude,double
1,latitude,double
2,housing_median_age,double
3,total_rooms,double
4,total_bedrooms,double
5,population,double
6,households,double
7,median_income,double
8,median_house_value,double
9,ocean_proximity,string


Ознакомимся с первыми 10 строками данных.

In [4]:
df.show(10)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

Посмотрим статистику по данным.

In [5]:
df.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


#### Выводы

Мы инициализировали сессию Spark, загрузили библиотеки и данные, провели осмотр данных.

### Подготовка данных

Исследуем данные на наличие пропусков.

In [6]:
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|        0|       0|                 0|          0|           207|         0|         0|            0|                 0|              0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+



Заполним их медианными значениями.

In [7]:
imputer = Imputer(strategy='median', inputCols=['total_bedrooms'], outputCols=['total_bedrooms'])
df = imputer.fit(df).transform(df)

Проверим статистику.

In [8]:
df.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20640,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,536.8388565891473,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,419.39187792168866,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


Среднее и стандартное отклонение почти не изменились.

#### Выводы

Проверили пропуски в данных и заполнили их медианными значениями.

### Преобразование категориальных признаков

Преобразуем колонку с категориальными значениями техникой One hot encoding.

Разделим колонки на два типа: числовые и текстовые, которые представляют категориальные данные.

In [9]:
categorical_cols = ['ocean_proximity']
numerical_cols  = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
                   'total_bedrooms', 'population', 'households', 'median_income']
target = 'median_house_value' 

Применим технику сначала StringIndexer для перевода категориальных признаков в численные.

In [10]:
indexer = StringIndexer(inputCols=categorical_cols, 
                        outputCols=[c+'_idx' for c in categorical_cols]) 
df = indexer.fit(df).transform(df)

cols = [c for c in df.columns for i in categorical_cols if (c.startswith(i))]
df.select(cols).show(3)

+---------------+-------------------+
|ocean_proximity|ocean_proximity_idx|
+---------------+-------------------+
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
+---------------+-------------------+
only showing top 3 rows



Теперь можно применить технику OHE.

In [11]:
encoder = OneHotEncoder(inputCols=[c+'_idx' for c in categorical_cols],
                        outputCols=[c+'_ohe' for c in categorical_cols])
df = encoder.fit(df).transform(df)

cols = [c for c in df.columns for i in categorical_cols if (c.startswith(i))]
df.select(cols).show(3)

+---------------+-------------------+-------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------------+-------------------+-------------------+
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
+---------------+-------------------+-------------------+
only showing top 3 rows



#### Выводы

Перевели в векторный вид категориальную колонку.

### Преобразование числовых признаков

Для числовых признаков тоже нужна трансформация — шкалирование значений — чтобы сильные выбросы не смещали предсказания модели. Применим StandardScaler.

Соберём вектор числовых признаков в отдельный столбец.

In [12]:
numerical_assembler = VectorAssembler(inputCols=numerical_cols, outputCol="numerical_features")
df = numerical_assembler.transform(df) 

Применим к столбцу с вектором числовых признаков StandardScaler.

In [13]:
standardScaler = StandardScaler(inputCol='numerical_features', outputCol="numerical_features_scaled")
df = standardScaler.fit(df).transform(df)
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- ocean_proximity_idx: double (nullable = false)
 |-- ocean_proximity_ohe: vector (nullable = true)
 |-- numerical_features: vector (nullable = true)
 |-- numerical_features_scaled: vector (nullable = true)



Финальный шаг — собрать трансформированные категорийные и числовые признаки с помощью VectorAssembler.

In [14]:
all_features = ['ocean_proximity_ohe','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features, outputCol='features')
df = final_assembler.transform(df)

In [15]:
df.select('features').show(3)
df.select('numerical_features_scaled').show(3)

+--------------------+
|            features|
+--------------------+
|[0.0,0.0,0.0,1.0,...|
|[0.0,0.0,0.0,1.0,...|
|[0.0,0.0,0.0,1.0,...|
+--------------------+
only showing top 3 rows

+-------------------------+
|numerical_features_scaled|
+-------------------------+
|     [-61.007269596069...|
|     [-61.002278409814...|
|     [-61.012260782324...|
+-------------------------+
only showing top 3 rows



#### Выводы

Мы преобразовали числовые признаки в вектора, объединили с категориальным и создали два набора данных. Один включающий все признаки, второй только количественные.

## Обучение моделей

Разделим наши наборы данных на тренировочные и тестовые.

In [16]:
train, test = df.randomSplit([.8,.2], seed=SEED)
print(train.count(), test.count())

16573 4067


Создадим и обучим оценщик LinearRegression.

In [17]:
lr_all = LinearRegression(labelCol=target, featuresCol='features',
                          maxIter=10, regParam=0.3, elasticNetParam=0.8)
model_all = lr_all.fit(train)

lr_num = LinearRegression(labelCol=target, featuresCol='numerical_features_scaled',
                          maxIter=10, regParam=0.3, elasticNetParam=0.8)
model_num = lr_num.fit(train)

22/08/02 19:37:04 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/08/02 19:37:04 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


Посмотрим на предсказания моделей, со всеми признаками:

In [18]:
predictions_all = model_all.transform(test)

predicted_all = predictions_all.select('median_house_value', 'prediction')
predicted_all.show(10)

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          106700.0|231885.93584759417|
|           78300.0|155799.70800570352|
|           70000.0|166422.57695611543|
|           70500.0|161990.87112883944|
|           86400.0|200405.75004113186|
|           60000.0|    159650.6852666|
|          105900.0|164879.29050001758|
|          109400.0| 200361.7137626009|
|           85100.0| 193512.0517316456|
|           92700.0| 202673.4666285324|
+------------------+------------------+
only showing top 10 rows



И только с числовыми признаками:

In [19]:
predictions_num = model_num.transform(test)

predicted_num = predictions_num.select('median_house_value', 'prediction')
predicted_num.show(10)

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          106700.0|192166.85561590036|
|           78300.0| 85574.56148346001|
|           70000.0|114982.99945447873|
|           70500.0| 110024.1406641244|
|           86400.0|153491.36768194614|
|           60000.0|110568.83214117633|
|          105900.0| 88779.07390948059|
|          109400.0|122012.70780267939|
|           85100.0|148191.54561168514|
|           92700.0|154945.25986321922|
+------------------+------------------+
only showing top 10 rows



### Выводы

Мы разделили данные на обучающие и тестовые выборки, обучили модели линейной регрессии и посмотрели на их предсказания.

## Анализ результатов

Сравним результаты работы линейной регрессии на двух наборах данных по метрикам RMSE, MAE и R2.

In [20]:
training_summary_all = model_all.summary
print('RMSE: %f' % training_summary_all.rootMeanSquaredError)
print('MAE: %f' % training_summary_all.meanAbsoluteError)
print('R2: %f' % training_summary_all.r2)

RMSE: 69003.554642
MAE: 50038.513122
R2: 0.640441


In [21]:
training_summary_num = model_num.summary
print('RMSE: %f' % training_summary_num.rootMeanSquaredError)
print('MAE: %f' % training_summary_num.meanAbsoluteError)
print('R2: %f' % training_summary_num.r2)

RMSE: 69806.260737
MAE: 50949.855170
R2: 0.632027


RMSE и MAE измеряют разницу между предсказанным значением медианной цены дома и его реальной ценой. Реальная цена по набору лежит в диапазоне от 15000 до 500000. Ошибки в случае с использованием всего набора данных чуть меньше. Но не существенно.

Коэффициент детерминации, показывающий долю корректных предсказаний нашей модели выше у модели учитывающей все признаки.

Проверим наши модели на тестовых данных.

In [22]:
test_summary_all = model_all.evaluate(test)
print('RMSE: %f' % test_summary_all.rootMeanSquaredError)
print('MAE: %f' % test_summary_all.meanAbsoluteError)
print('R2: %f' % test_summary_all.r2)

RMSE: 70940.383666
MAE: 50393.890118
R2: 0.630216


In [23]:
test_summary_num = model_num.evaluate(test)
print('RMSE: %f' % test_summary_num.rootMeanSquaredError)
print('MAE: %f' % test_summary_num.meanAbsoluteError)
print('R2: %f' % test_summary_num.r2)

RMSE: 71560.050901
MAE: 51063.192068
R2: 0.623728


### Выводы

Мы проверили результаты работы линейной регрессии на обучающих и тестовых наборах данных.

Вычислили по три метрики для всех результатов: `RMSE`, `MAE`, `R2`.

В целом модели показывают неплохой результат. На тестовом наборе данных лучший результат показала модель обученная как на количественных так и на категориальном признаках.

На тестовом наборе все метрики ухудшились, но незначительно. Можем считать что модели не переобучились.

## Общий вывод

В данном проекте мы обучили модель линейной регрессии на данных о жилье в Калифорнии в 1990 году.

Научились инициализировать `Spark` сессии, загружать данные и обрабатывать их с помощью DataFrame API и работать с моделями в библиотеке MLlib (DataFrame-based).

В целом полученные модели показывают неплохой результат. На тестовом наборе данных лучший результат показала модель обученная как на количественных так и на категориальном признаках. На тестовом наборе все метрики ухудшились, но незначительно. Можем считать что модели не переобучились.

In [24]:
spark.stop()